# Advent of Code
## Day 7 - [No Space Left On Device](https://adventofcode.com/2022/day/7)

### Step 1: Parse input into list of commands

In [189]:
def get_inputs(filename): 
    with open(filename) as file: 
        return file.read().strip()
        
def parse_cmd_args(args_input): return [arg.split(' ') for arg in args_input.strip().split('\n')]
def parse_cmd(cmd_input):       return [cmd_input[:2], parse_cmd_args(cmd_input[2:])]
def parse_cmds(inputs):         return [parse_cmd(i) for i in inputs.split('$ ') if i]

### Step 2: 'run' commands to reveal the file system tree

In [190]:
ROOT_PATH, PATH_SEP = '', '/'

def start():
    root = { 'path': ROOT_PATH, 'directories':set(), 'files':set() }
    return ({ ROOT_PATH: root },root)

def path(item): return item['path']

def cd(index, current_dir, dir_name):
    if dir_name == "..":  
        current_path = path(current_dir)
        parent_path = current_path[:current_path.rfind(PATH_SEP)]
        return index[parent_path]
    elif dir_name == "/": 
        return index[ROOT_PATH]
    else:
        result = { 'path': f'{path(current_dir)}{PATH_SEP}{dir_name}', 'directories':set(), 'files':set() }
        index[path(result)] = result
        current_dir['directories'].add(path(result))
        return result

In [191]:
# PARSE
assert parse_cmds(get_inputs('../data/Day07-part1.txt')) == [
    ['cd', [['/']]],
    ['ls', [['dir', 'a'], ['14848514', 'b.txt'], ['8504156', 'c.dat'], ['dir', 'd']]],
    ['cd', [['a']]],
    ['ls', [['dir', 'e'], ['29116', 'f'], ['2557', 'g'], ['62596', 'h.lst']]],
    ['cd', [['e']]],
    ['ls', [['584', 'i']]],
    ['cd', [['..']]],
    ['cd', [['..']]],
    ['cd', [['d']]],
    ['ls', [['4060174', 'j'], ['8033020', 'd.log'], ['5626152', 'd.ext'], ['7214296', 'k']]]
], 'part 1 test input'

# CD
index, parent = start()

child = cd(index, parent, "b")
grandchild = cd(index, child, "c")

assert parent == { 'path': '', 'directories':{'/b'}, 'files':set()}, 'cd updates parent directory set with child path'
assert child  == { 'path': '/b', 'directories':{'/b/c'}, 'files':set()}, 'cd returns directory with correct path'
assert grandchild  == { 'path': '/b/c', 'directories':set(), 'files':set()}, 'cd returns directory with correct path'
assert index  == { '': parent, '/b': child, '/b/c': grandchild }, 'cd adds new child to path index'
assert cd(index, grandchild, "..") == child, 'cd .. move up a directory'
assert cd(index, grandchild, "/" ) == parent, 'cd / returns root'

